In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install Pillow

In [ ]:
pip install rembg

Add border and resize

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from rembg import remove
from PIL import Image
from PIL import ImageOps
import os

def process_image(image_path):
    original_image = Image.open(image_path)

    target_size = (1920, 2400)

    width, height = original_image.size
    aspect_ratio = target_size[0] / target_size[1]

    if (width / height) > aspect_ratio:
        new_width = int(height * aspect_ratio)
        left = (width - new_width) // 2
        top = 0
        right = left + new_width
        bottom = height
    else:
        new_height = int(width / aspect_ratio)
        top = (height - new_height) // 2
        left = 0
        right = width
        bottom = top + new_height

    object_image = original_image.crop((left, top, right, bottom))

    border_size = 20

    new_width = object_image.width + 2 * border_size
    new_height = object_image.height + 2 * border_size

    new_background = Image.new('RGB', (new_width, new_height), white_color)

    x_offset = border_size
    y_offset = border_size

    new_background.paste(object_image, (x_offset, y_offset))

    final_image = new_background.resize(target_size, Image.ANTIALIAS)


    relative_path = os.path.relpath(image_path, input_folder)


    output_subfolder = os.path.dirname(relative_path)
    output_folder_path = os.path.join(output_folder, output_subfolder)

    os.makedirs(output_folder_path, exist_ok=True)

    output_file_path = os.path.join(output_folder_path, os.path.splitext(os.path.basename(image_path))[0] + '.jpg')

    final_image.save(output_file_path, 'JPEG')

input_folder = '/content/drive/MyDrive/input2/'
output_folder = '/content/drive/MyDrive/output22/'
os.makedirs(output_folder, exist_ok=True)

white_color = (255, 255, 255)

image_paths = []

for root, dirs, files in os.walk(input_folder):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png', '.webp','.jfif','.PNG','.JPG','.WEBP','.JFIF')):
            image_path = os.path.join(root, filename)
            image_paths.append(image_path)


with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    executor.map(process_image, image_paths)

In [ ]:
#only resize
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import os

def process_image(image_path):
    original_image = Image.open(image_path)

    target_size = (1920, 2400)

    final_image = original_image.resize(target_size, Image.ANTIALIAS)

    relative_path = os.path.relpath(image_path, input_folder)

    output_subfolder = os.path.dirname(relative_path)
    output_folder_path = os.path.join(output_folder, output_subfolder)

    os.makedirs(output_folder_path, exist_ok=True)


    output_file_path = os.path.join(output_folder_path, os.path.splitext(os.path.basename(image_path))[0] + '.jpg')

    final_image.save(output_file_path, 'JPEG')

input_folder = '/content/drive/MyDrive/input2/'
output_folder = '/content/drive/MyDrive/output22/'
os.makedirs(output_folder, exist_ok=True)

image_paths = []

for root, dirs, files in os.walk(input_folder):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png', '.webp', '.jfif', '.PNG', '.JPG', '.WEBP', '.JFIF')):
            image_path = os.path.join(root, filename)
            image_paths.append(image_path)

# Use a ThreadPoolExecutor to process images in parallel
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    executor.map(process_image, image_paths)



Remove background

In [ ]:
#remove background of images

from rembg import remove
from PIL import Image
from PIL import ImageOps
import os

input_directory = '/content/drive/MyDrive/input2/'
output_directory = '/content/drive/MyDrive/output22/'


os.makedirs(output_directory, exist_ok=True)

for filename in os.listdir(input_directory):
    if filename.lower().endswith(('.jpg', '.png','.webp','.jfif')):
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)

        input_image = Image.open(input_path)

        output = remove(input_image, alpha_matting=True)

        background_color = (255, 255, 255)
        new_image = Image.new("RGB", output.size, background_color)
        new_image.paste(output, (0, 0), output)

        new_image.save(output_path, format='png')

print("Processing complete")

increase size of object in image

In [ ]:
#increase size of object in image

import cv2
import os

input_folder = "/content/drive/MyDrive/output22/"
output_folder = "/content/drive/MyDrive/output22_processed/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

connectivity = 8

for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_folder, filename)

        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        output = cv2.connectedComponentsWithStats(thresh, connectivity, cv2.CV_32S)
        (numLabels, labels, stats, centroids) = output

        for i in range(0, numLabels):
            if i == 0:
                continue

            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]

            increase_factor = 1.3
            new_w = int(w * increase_factor)
            new_h = int(h * increase_factor)

            resized_object = cv2.resize(image[y:y+h, x:x+w], (new_w, new_h))
            x_offset = x + (w - new_w) // 2
            y_offset = y + (h - new_h) // 2
            image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_object


        output_image_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_image_path, image)
